In [1]:
from src.data import read_raw_data, preprocess_data

data = read_raw_data()
data = preprocess_data(data)

[Pipeline]  (step 1 of 3) Processing functiontransformer-1, total=   0.0s
[Pipeline]  (step 2 of 3) Processing functiontransformer-2, total=   0.0s
[Pipeline]  (step 3 of 3) Processing functiontransformer-3, total=   0.0s


In [2]:
from src.features import add_num_multirows

print(data['train'].shape)

data['train'] = add_num_multirows(data['train'], split='train')
data['test'] = add_num_multirows(data['test'], split='test')

print(data['train'].shape)

(9928, 12)
(7918, 12)


In [3]:
from src.features import add_num_multirows, SpeciesEncoder, TrapFeatureExtractor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer


species_oh_encoder = SpeciesEncoder()
trap_feat_extractor = TrapFeatureExtractor()

feature_pipeline = make_pipeline(
    species_oh_encoder,
    trap_feat_extractor
)

data['train'] = feature_pipeline.fit_transform(data['train'])
data['test'] = feature_pipeline.transform(data['test'])


In [4]:
from sklearn.preprocessing import KBinsDiscretizer


lat_enc = KBinsDiscretizer(n_bins=5, encode='ordinal')

data['train']['lat_bin'] = lat_enc.fit_transform(data['train'][['Latitude']])
data['test']['lat_bin'] = lat_enc.transform(data['test'][['Latitude']])

long_enc = KBinsDiscretizer(n_bins=3, encode='ordinal')

data['train']['long_bin'] = long_enc.fit_transform(data['train'][['Longitude']])
data['test']['long_bin'] = long_enc.transform(data['test'][['Longitude']])

In [5]:
data['train']

,Date,Trap,NumRows,NumMosquitos,WnvPresent,Latitude,Longitude,Month,Year,Week,Dayofyear,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,trap_max_mos,trap_wnv_proba,lat_bin,long_bin
0,2007-05-29,T002,1,1,0,41.954690,-87.800991,5,2007,22,149,0.0,1.0,0.0,582,0.181818,3.0,0.0
1,2007-05-29,T002,1,1,0,41.954690,-87.800991,5,2007,22,149,0.0,0.0,1.0,582,0.181818,3.0,0.0
2,2007-05-29,T007,1,1,0,41.994991,-87.769279,5,2007,22,149,0.0,0.0,1.0,18,0.000000,4.0,0.0
3,2007-05-29,T015,1,1,0,41.974089,-87.824812,5,2007,22,149,0.0,1.0,0.0,39,0.235294,4.0,0.0
4,2007-05-29,T015,1,4,0,41.974089,-87.824812,5,2007,22,149,0.0,0.0,1.0,39,0.235294,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7913,2013-09-26,T232,1,1,0,41.912563,-87.668055,9,2013,39,269,0.0,1.0,0.0,72,0.064516,3.0,1.0
7914,2013-09-26,T233,1,5,0,42.009876,-87.807277,9,2013,39,269,0.0,1.0,0.0,200,0.160000,4.0,0.0
7915,2013-09-26,T235,1,1,0,41.776428,-87.627096,9,2013,39,269,0.0,1.0,0.0,93,0.166667,1.0,2.0
7916,2013-09-26,T900,4,37,0,41.974689,-87.890615,9,2013,39,269,1.0,0.0,0.0,1596,0.377358,4.0,0.0


In [6]:
train_columns = ['Week', 'Dayofyear', 'Latitude', 'Longitude',
       'trap_max_mos', 'trap_wnv_proba', 'NumRows',#'lat_bin', 'long_bin',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS', 'WnvPresent']

test_columns = ['Id', 'Week', 'Dayofyear', 'Latitude', 'Longitude',
       'trap_max_mos', 'trap_wnv_proba', 'NumRows', #'lat_bin', 'long_bin',
       'Species_CULEX PIPIENS', 'Species_CULEX PIPIENS/RESTUANS',
       'Species_CULEX RESTUANS']

In [7]:
data['train'] = data['train'].merge(data['weather'], on='Date')
data['test'] = data['test'].merge(data['weather'], on='Date')

In [8]:
data['weather'].columns

Index(['Date', 'Tmax_1', 'Tmin_1', 'Tavg_1', 'DewPoint_1', 'WetBulb_1',
       'PrecipTotal_1', 'AvgSpeed_1', 'ResultSpeed_1', 'ResultDir_1', 'Tmax_2',
       'Tmin_2', 'Tavg_2', 'DewPoint_2', 'WetBulb_2', 'PrecipTotal_2',
       'AvgSpeed_2', 'ResultSpeed_2', 'ResultDir_2'],
      dtype='object')

In [9]:
data['weather'][['ResultDir_1', 'ResultDir_2']].corr()

,ResultDir_1,ResultDir_2
ResultDir_1,1.000000,0.822797
ResultDir_2,0.822797,1.000000


In [10]:
weather_columns = ['Tmax_1', 'Tmin_1', 'Tavg_1', 'DewPoint_1', 'WetBulb_1',
       'PrecipTotal_1', 'AvgSpeed_1', 'PrecipTotal_2',
       #'AvgSpeed_2'] #, 'ResultSpeed_2', 'ResultDir_2' 'ResultDir_1', 'ResultSpeed_1'
       ]

In [11]:
AGG_COLS = ['Tmax_1','Tmin_1','PrecipTotal_1',
            'DewPoint_1', 'AvgSpeed_1', 'PrecipTotal_2', 'Tavg_1'
            # WetBulb_1', , 
       ]

LAG_LIST = [1, 8, 15]
WINDOW_LIST = [7]
AGG_LIST = ['mean']

In [12]:
from src.features import aggregate_columns_with_lag


df_agg = aggregate_columns_with_lag(
    data['weather'],
    columns=AGG_COLS,
    lags=LAG_LIST,
    windows=WINDOW_LIST,
    agg_func=AGG_LIST
).reset_index()

In [13]:
data['train'] = data['train'].merge(df_agg, on='Date')
data['test'] = data['test'].merge(df_agg, on='Date')

In [14]:
agg_weather_columns = df_agg.columns.to_list()[1:]

In [15]:
X_train = data['train'][train_columns+weather_columns+agg_weather_columns].copy()
y_train = X_train.pop('WnvPresent')

In [16]:
# from sklearn.feature_selection import SequentialFeatureSelector
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier

# # selector_model = LGBMClassifier(verbose=-1)
# selector_model = XGBClassifier()

# X_new = X_train[weather_columns+agg_weather_columns].copy()

# selector = SequentialFeatureSelector(
#     selector_model,
#     n_features_to_select=10,
#     direction='forward'
# ).fit(X_new, y_train)

# support = selector.support_
# selected_features = X_new.columns[support].to_list()
# print(selected_features)


In [17]:
# X_train = X_train[train_columns[:-1]+selected_features]

In [24]:
from src.data import read_raw_data, preprocess_data
from src.features import get_features


data = read_raw_data()
data = preprocess_data(data)
train_df, test_df = get_features(data)

[Pipeline]  (step 1 of 3) Processing functiontransformer-1, total=   0.0s
[Pipeline]  (step 2 of 3) Processing functiontransformer-2, total=   0.0s
[Pipeline]  (step 3 of 3) Processing functiontransformer-3, total=   0.0s


2024-10-01 20:08:52.673 | DEBUG    | src.features:get_features:146 - Aggregating weather with lag...
2024-10-01 20:08:52.727 | INFO     | src.features:get_features:154 - Weather aggregated and lagged.


In [27]:
X_train = train_df.copy()
y_train = X_train.pop('WnvPresent')

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold

from lightgbm import LGBMClassifier


scaler = StandardScaler()
# clf = LogisticRegression(C=0.01)
clf = LGBMClassifier(reg_lambda=50, reg_alpha=1, colsample_bytree=0.8) 

steps = [
    ('scale', scaler),
    ('classifier', clf)
]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)

results_cv = cross_validate(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')

print(results_cv['test_score'].mean())

[LightGBM] [Info] Number of positive: 364, number of negative: 5970
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044842 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3149
[LightGBM] [Info] Number of data points in the train set: 6334, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.057468 -> initscore=-2.797348
[LightGBM] [Info] Start training from score -2.797348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 364, number of negative: 5970
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001510 seconds.
You can set `force_row_wise=true` to remove the o

In [29]:
pipeline.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 455, number of negative: 7463
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001731 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3171
[LightGBM] [Info] Number of data points in the train set: 7918, number of used features: 59
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.057464 -> initscore=-2.797415
[LightGBM] [Info] Start training from score -2.797415


Pipeline(steps=[('scale', StandardScaler()),
                ('classifier',
                 LGBMClassifier(colsample_bytree=0.8, reg_alpha=1,
                                reg_lambda=50))])

In [30]:
X_test = test_df.copy()
# X_test = data['test'][test_columns+weather_columns+agg_weather_columns].copy()
# X_test = data['test'][test_columns +selected_features]

y_pred = pipeline.predict_proba(X_test.drop('Id', axis=1))[:,1]
X_test['pred_proba'] = y_pred

predictions_df = X_test[['Id', 'pred_proba']].copy()
predictions_df

,Id,pred_proba
0,1,0.006060
1,2,0.005846
2,3,0.006205
3,9,0.003099
4,10,0.003099
...,...,...
43632,115071,0.005463
43633,115072,0.005463
43634,115078,0.017410
43635,115079,0.013193


In [31]:
import pandas as pd


sample = pd.read_csv('../data/raw/sampleSubmission.csv')
results = pd.merge(sample, predictions_df, on='Id', how='outer').fillna(0)

results['WnvPresent'] = results['pred_proba']
results.drop('pred_proba', axis=1, inplace=True)

results.to_csv('../data/submission/test_results.csv', index=False)

In [32]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

api.competition_submit('../data/submission/test_results.csv', 'test_lr', 'predict-west-nile-virus')

100%|██████████| 1.81M/1.81M [00:01<00:00, 1.15MB/s]


Successfully submitted to West Nile Virus Prediction

In [33]:
api.competitions_submissions_list(id='predict-west-nile-virus')[0]['privateScore']

'0.78194'

In [29]:
X_train.columns

Index(['Week', 'Dayofyear', 'Latitude', 'Longitude', 'trap_max_mos',
       'trap_wnv_proba', 'Species_CULEX PIPIENS',
       'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS', 'Tmax_1',
       'Tmin_1', 'Tavg_1', 'DewPoint_1', 'WetBulb_1', 'PrecipTotal_1',
       'AvgSpeed_1', 'Tmax_1_mean_l1_w7', 'Tmin_1_mean_l1_w7',
       'PrecipTotal_1_mean_l1_w7', 'DewPoint_1_mean_l1_w7',
       'AvgSpeed_1_mean_l1_w7', 'Tmax_1_mean_l8_w7', 'Tmin_1_mean_l8_w7',
       'PrecipTotal_1_mean_l8_w7', 'DewPoint_1_mean_l8_w7',
       'AvgSpeed_1_mean_l8_w7', 'Tmax_1_mean_l15_w7', 'Tmin_1_mean_l15_w7',
       'PrecipTotal_1_mean_l15_w7', 'DewPoint_1_mean_l15_w7',
       'AvgSpeed_1_mean_l15_w7'],
      dtype='object')